In [29]:
import  requests
import json
import pandas as pd  
import os 
import mimetypes
import time 


In [2]:
job_dir = "JobDesc"
resume_dir = "Resumes"

In [3]:
urls = {
    "upload_resume" : "https://avermass-cv-ranking.com/uploadfiles/resumes",
    "upload_jobs" : "https://avermass-cv-ranking.com/uploadfiles/jobbdesc",
    "similarity_ranking" : "https://avermass-cv-ranking.com/similarity-rankings"
}

In [31]:
def upload_resume(client_id , resume_path, max_tries = 10):
  payload={}
  mime_type, _ = mimetypes.guess_type(resume_path)

  # If the MIME type cannot be determined, set it to 'application/octet-stream'
  if mime_type is None:
      mime_type = 'application/octet-stream'
  files=[
    (
        'files',
        (
          os.path.basename(resume_path),
          open(resume_path,'rb'),
          mime_type
        )
        ),
    ]
  headers = {}
  url = urls["upload_resume"] + f"?client_id={client_id}"
  for i in range(max_tries):
    response = requests.request("POST", url, headers=headers, data=payload, files=files)
    if response.status_code == 200:
      return True
    time.sleep(1)
    print("Retrying ", resume_path)
  print(f"Error uploading {resume_path}", response.status_code, mime_type )
  return False


In [32]:
def upload_jobs(job_path, max_tries = 10):
  payload={}
  mime_type, _ = mimetypes.guess_type(job_path)

  # If the MIME type cannot be determined, set it to 'application/octet-stream'
  if mime_type is None:
      mime_type = 'application/octet-stream'
  files=[
    (
        'files',
        (
          os.path.basename(job_path),
          open(job_path,'rb'),
          mime_type
        )
        ),
    ]
  headers = {}
  url = urls["upload_jobs"] 
  for i in range(max_tries):
    response = requests.request("POST", url, headers=headers, data=payload, files=files)
    if response.status_code == 200:
      return response.json()["client_id"]
    time.sleep(1)
    print("Retrying ", job_path)
  print(f"Error uploading {job_path}", response.status_code, mime_type)
  return False

In [33]:
def get_similarity_ranking(client_id, index = 0 ):
  url = f"https://avermass-cv-ranking.com/similarity-rankings?index={index}&client_id={client_id}"

  payload={'index': '0'}
  files=[

  ]
  headers = {}

  response = requests.request("POST", url, headers=headers, data=payload, files=files)
  if response.status_code == 200:
    return response.json()
  return False

In [34]:
df = []
for job in [os.listdir(job_dir)[0]]:
    job_path = os.path.join(
            os.getcwd(),
            job_dir,
            job
        )
    for resume in os.listdir(resume_dir):

        resume_path = os.path.join(
            os.getcwd(),
            resume_dir,
            resume
        )


        to_append = {
                "jobs" : job_path,
                "Resume File" : resume,
                "Status" : False,
                "Response" : "",
                "client_id" : ""
            }
        client_id = upload_jobs(job_path)
        if client_id:
            to_append["client_id"] = client_id
            if upload_resume(client_id, resume_path, 10):
                results = get_similarity_ranking(client_id=client_id)
                if results:
                    to_append["Status"] =True
                    to_append["Response"] = str(results)
        print(to_append)
        df.append(to_append)


{'jobs': '/home/vk001716/ResumeRanking-NLP/Tests/JobDesc/12.docx', 'Resume File': '67.pdf', 'Status': True, 'Response': "[{'Name': 'Data/Resumes/08118f0b-b4a9-42fb-8873-3861faea609f/67.docx', 'scores': 51.4656845934, 'rank': 1}]", 'client_id': '08118f0b-b4a9-42fb-8873-3861faea609f'}
{'jobs': '/home/vk001716/ResumeRanking-NLP/Tests/JobDesc/12.docx', 'Resume File': '59.docx', 'Status': False, 'Response': '', 'client_id': 'f8fd15a1-4deb-4254-a6d0-fa001e31519e'}
Retrying  /home/vk001716/ResumeRanking-NLP/Tests/JobDesc/12.docx
Retrying  /home/vk001716/ResumeRanking-NLP/Tests/JobDesc/12.docx
Retrying  /home/vk001716/ResumeRanking-NLP/Tests/JobDesc/12.docx
Retrying  /home/vk001716/ResumeRanking-NLP/Tests/JobDesc/12.docx
Retrying  /home/vk001716/ResumeRanking-NLP/Tests/JobDesc/12.docx
{'jobs': '/home/vk001716/ResumeRanking-NLP/Tests/JobDesc/12.docx', 'Resume File': '33.pdf', 'Status': False, 'Response': '', 'client_id': '6c0ad0c0-b63a-4dda-ae8a-9c008fe47bf3'}
{'jobs': '/home/vk001716/ResumeRan

In [ ]:
df_ = pd.DataFrame(df)

In [ ]:
df_[df_.Status == False].to_csv("failed_results.csv")

In [ ]:
df_[df_.Status == False]

,jobs,Resume File,Status,Response,client_id
1,/home/vk001716/ResumeRanking-NLP/Tests/JobDesc...,59.docx,False,,72385283-ca9a-4167-9dcd-575d34f4532c
2,/home/vk001716/ResumeRanking-NLP/Tests/JobDesc...,33.pdf,False,,
3,/home/vk001716/ResumeRanking-NLP/Tests/JobDesc...,22.pdf,False,,
4,/home/vk001716/ResumeRanking-NLP/Tests/JobDesc...,60.pdf,False,,
5,/home/vk001716/ResumeRanking-NLP/Tests/JobDesc...,71.pdf,False,,
6,/home/vk001716/ResumeRanking-NLP/Tests/JobDesc...,30.pdf,False,,
7,/home/vk001716/ResumeRanking-NLP/Tests/JobDesc...,43.pdf,False,,
8,/home/vk001716/ResumeRanking-NLP/Tests/JobDesc...,20.pdf,False,,
9,/home/vk001716/ResumeRanking-NLP/Tests/JobDesc...,72.pdf,False,,
10,/home/vk001716/ResumeRanking-NLP/Tests/JobDesc...,56.pdf,False,,
